In [ ]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

city_daily_temperature_df = pd.read_csv("Datasets/city-daily-temperature.csv")
global_country_info_df = pd.read_csv("Datasets/global-country-information.csv")
print(city_daily_temperature_df.shape)
print(global_country_info_df.shape)

: 

In [50]:
# Preprocessing City Daily Temperature

In [ ]:
# Preprocessing Global Country Information
global_country_info_df = global_country_info_df.drop(['Abbreviation','Calling Code','Currency-Code','Official language','Total tax rate', 'Physicians per thousand', 'Minimum wage', 'Out of pocket health expenditure',], axis = 1)
global_country_info_df[global_country_info_df.isnull().any(axis=1)]